In [27]:
import pandas as pd

beat = pd.read_csv('pd_beat_neighborhoods_datasd.csv')
call = pd.read_csv('pd_calls_for_service_2017_datasd.csv')
census = pd.read_csv('sandiegocensustract.csv')

In [28]:
# This is cleaning up the call df columns
call = call[['date_time','street','streettype','beat', 'priority']]


# This is cleaning up the census df 
census = census.drop('TRACTNUM', axis = 1)
census = census.drop('TRACT', axis = 1)
census = census.drop('SevCrwd', axis = 1)
census = census.drop('TeenBirthAllWom', axis = 1)
census = census.drop('MaltrtAllegRate', axis = 1)
census = census.drop('ProxOffAlco', axis = 1)
census = census.drop('ProxOnAlco', axis = 1)
census = census.drop('TraffInjur', axis = 1)
census = census.drop('SNAP_FdStmp', axis = 1)
census = census.drop('n_FoodDesert', axis = 1)
census = census.drop('pct_FoodDesert', axis = 1)
census = census.drop('LiqCount', axis = 1)
census = census.drop('VoterPartic', axis = 1)
census = census.drop('HghSchOrHigh', axis = 1)
census = census.drop('Uninsured', axis = 1)
census = census.drop('PovertyPctl', axis = 1)
census = census.drop('Longitude', axis = 1)
census = census.drop('Latitude', axis = 1)
census = census.drop('LowBirthWeight', axis = 1)
census = census.drop('LowBirthWeightPctl', axis = 1)
census = census.drop('PollutionBurdenPctl', axis = 1)
census = census.drop('CES20Score', axis = 1)
census = census.drop('SingMother', axis = 1)
census = census.drop('TeenBirthProportion', axis = 1)
census = census.drop('PollutionBurdenScore', axis = 1)
census = census.drop('CES20PercentileRange', axis = 1)
census = census.drop('PopCharScore', axis = 1)
census = census.drop('PopCharPctl', axis = 1)
census = census.drop('FosterCareEntry', axis = 1)
census = census.drop('TotalPov', axis = 1)
census = census.drop('ChildPov', axis = 1)
census = census.drop('Education', axis = 1)

In [29]:
# This is replacing date_time column to year
call['date_time'] = call['date_time'].str[:4]
call.columns = ['year','street','streettype','beat', 'priority']

In [30]:
# code that says, if beat of calldf is not in beat of beatdf, then remove that row

#call.sort_values(['beat'], ascending = True)
# This replaces all 0s and 999s in beat with NaN. 
# Must eventually say: if beat of calldf is not in beat of beatdf, drop that row
call['beat'] = call['beat'].replace(0,'NaN')
call['beat'] = call['beat'].replace(999,'Nan')
print (call.shape)
# This drops all rows where beat is 0
#call = call.loc[(call['beat'] != 'NaN')]
call = call['beat'].dropna(how='any')
type(call['beat'])

(218652, 5)


KeyError: 'beat'

In [21]:
call

pandas.core.frame.DataFrame

In [6]:
# This code wiill return a new df with total pop per city. maybe take average of pov,unemployment
# Maybe put all the zip codes associated with a city, into a dict

In [18]:
#print (census['ZIP'].unique().size)
print (census['NeighborhoodName'].unique().size)
census['City'].unique()

238


array(['San Diego', 'San Marcos', 'Oceanside', 'Escondido', 'Vista',
       'Carlsbad', 'El Cajon', 'Fallbrook', 'National City', 'La Mesa',
       'San Ysidro', 'Encinitas', 'Lakeside', 'Chula Vista',
       'Borrego Springs', 'Spring Valley', 'Poway', 'Santee',
       'Imperial Beach', 'Campo', 'Warner Springs', 'Solana Beach',
       'Lemon Grove', 'La Jolla', 'Valley Center', 'Ramona',
       'Pauma Valley', 'Bonita', 'Coronado', 'Alpine', 'Descanso', 'Jamul',
       'Cardiff By the Sea', 'Del Mar', 'Bonsall', 'Julian',
       'Rancho Santa Fe', 'Camp Pendleton', nan], dtype=object)

In [17]:
beat

,Beat,Neighborhood
0,111,Clairemont Mesa East
1,112,Clairemont Mesa West
2,113,Bay Ho
3,114,North Clairemont
4,115,University City
5,116,Bay Park
6,121,Mission Beach
7,122,Pacific Beach
8,123,Mission Bay Park
9,124,La Jolla
